# Working with Universal Dependencies

The Universal Dependencies framework distributes annotated corpora for many languages, all using the same dependency format. In this notebook, I'll demonstrate how to access such a corpus from within Python. 

We will work with English GUM corpus. You can find it listed under "English" on https://universaldependencies.org/#language- The repository with the actual corpus is at https://github.com/UniversalDependencies/UD_English-GUM/tree/master

From the github repository, please download the training portion of the corpus, and put it in the same directory as this notebook. 

First, we will take a look at the format in which Universal Dependencies corpora are stored.

In [30]:
# if you don't have conllu yet, uncomment the following
# !python3 -m pip install conllu

In [31]:
import conllu # reading Universal Dependency files in the CONLLu format

We open the GUM corpus as a text file, and look at its first few lines. After the initial metadata, the first sentence starts with the line
      "# text = Aesthetic Appreciation and Spanish Art:"

In [ ]:
with open("ko_kaist-ud-train.conllu", encoding="utf-8") as f:
    data = f.read()

In [35]:

def parse_conllu():
    sentences = []
    with open("ko_kaist-ud-train.conllu", encoding='utf-8') as fh:
        sent_lines = []
        for raw in fh:
            line = raw.rstrip('\n')
            if line.startswith('#'):
                sent_lines.append(line)
            elif line.strip() == '':
                if sent_lines:
                    sentences.append(sent_lines)
                    sent_lines = []
            else:
                sent_lines.append(line)
        if sent_lines:
            sentences.append(sent_lines)

    parsed = []
    for s in sentences:
        tokens = []
        for ln in s:
            if ln.startswith('#'):
                continue
            parts = ln.split('\t')
            if len(parts) != 10:
                continue
            id_, form, lemma, upos, xpos, feats, head, deprel, deps, misc = parts
            if '-' in id_:
                continue
            try:
                id_int = int(id_)
            except Exception:
                continue
            token = {
                'id': id_int,
                'form': form,
                'lemma': lemma,
                'upos': upos,
                'xpos': xpos,
                'feats': feats,
                'head': int(head) if head != '_' else None,
                'deprel': deprel,
                'deps': deps,
                'misc': misc
            }
            tokens.append(token)
        if tokens:
            tokens.sort(key=lambda t: t['id'])
            parsed.append(tokens)
    return parsed

sents = parse_conllu()
print(f'Parsed {len(sents)} sentences')

Parsed 23010 sentences


In [ ]:
import pandas as pd
import os
OUT_DIR = 'data/project_outputs'
os.makedirs(OUT_DIR, exist_ok=True)

from collections import defaultdict, Counter
verb_instances = 0
obj_instances = 0
obj_immediate = 0
obj_within3 = 0
dist_counter = Counter()

examples = defaultdict(list) 

for sent_idx, tokens in enumerate(sents):
    id_to_tok = {t['id']: t for t in tokens}
    dependents = defaultdict(list)
    for t in tokens:
        if t['head'] is not None and t['head'] in id_to_tok:
            dependents[t['head']].append(t)
    id_to_index = {t['id']: i for i, t in enumerate(tokens)}

    for i, tok in enumerate(tokens):
        if tok['upos'] == 'VERB':
            verb_instances += 1
            verb_id = tok['id']
            deps = dependents.get(verb_id, [])
            for dep in deps:
                if dep['deprel'] == 'obj':
                    obj_instances += 1
                    obj_idx = id_to_index.get(dep['id'])
                    verb_idx = i
                    if obj_idx is None:
                        continue
                    diff = verb_idx - obj_idx
                    dist_counter[diff] += 1
                    if diff == 1:
                        obj_immediate += 1
                    if 0 <= diff <= 1:
                        obj_within3 += 1
                    lemma = tok['lemma'] if tok['lemma'] != '_' else tok['form']
                    if len(examples[lemma]) < 5:
                        sent_form = ' '.join([t['form'] for t in tokens])
                        examples[lemma].append((dep['form'], tok['form'], sent_form))

if obj_instances > 0:
    pct_immediate = obj_immediate / obj_instances * 100
    pct_within3 = obj_within3 / obj_instances * 100
else:
    pct_immediate = pct_within3 = 0.0

print('Verb instances (VERB tokens):', verb_instances)
print('Total obj instances:', obj_instances)
print(f'Objects immediately before verb: {obj_immediate} ({pct_immediate:.2f}%)')
print(f'Objects within 1 tokens before verb: {obj_within3} ({pct_within3:.2f}%)')

dist_items = sorted(dist_counter.items())
df_dist = pd.DataFrame(dist_items, columns=['verb_minus_obj_index', 'count'])
df_dist.to_csv(os.path.join(OUT_DIR, 'object_verb_distance_distribution.csv'), index=False)
print('Saved distance distribution to', os.path.join(OUT_DIR, 'object_verb_distance_distribution.csv'))


# parse words within words
def num_words_in_word(word):
    length = len(word.split('+'))
    return length

# analyze how many words within words by part of speech
pos_wordcount = defaultdict(list)
for tokens in sents:
    for t in tokens:
        pos = t['upos']
        word = t['lemma']
        count = num_words_in_word(word)
        pos_wordcount[pos].append(count)
import numpy as np
pos_wordcount_stats = {}
for pos, counts in pos_wordcount.items():
    arr = np.array(counts)
    pos_wordcount_stats[pos] = {
        'mean': np.mean(arr),
        'median': np.median(arr),
        'max': np.max(arr),
        'min': np.min(arr),
        'count': len(arr)
    }
df_wordcount = pd.DataFrame.from_dict(pos_wordcount_stats, orient='index')
df_wordcount.to_csv(os.path.join(OUT_DIR, 'pos_wordcount_stats.csv'))
print('Saved POS word count stats to', os.path.join(OUT_DIR, 'pos_wordcount_stats.csv'))
print(df_wordcount)


Verb instances (VERB tokens): 55805
Total obj instances: 13912
Objects immediately before verb: 10066 (72.35%)
Objects within 1 tokens before verb: 10066 (72.35%)
Saved distance distribution to data/project_outputs/object_verb_distance_distribution.csv
Saved POS word count stats to data/project_outputs/pos_wordcount_stats.csv


In [ ]:
print(data[:4000])

# newdoc id = GUM_academic_art
# global.Entity = GRP-etype-infstat-salience-centering-minspan-link-identity
# meta::author = Claire Bailey-Ross, Andrew Beresford, Daniel Smith, Claire Warwick
# meta::dateCollected = 2017-09-13
# meta::dateCreated = 2017-08-08
# meta::dateModified = 2017-09-13
# meta::genre = academic
# meta::salientEntities = 4 (5*), 5 (5*), 44 (5*), 45 (5*), 46 (5*), 47 (5*), 27 (4*), 147 (4*), 2 (3*), 43 (3), 20 (2*), 23 (2), 63 (2), 72 (2), 73 (2), 3 (1), 19 (1), 24 (1), 26 (1), 48 (1), 49 (1), 50 (1), 62 (1), 68 (1), 69 (1), 74 (1), 76 (1), 77 (1), 78 (1), 79 (1), 158 (1)
# meta::sourceURL = https://dh2017.adho.org/abstracts/333/333.pdf
# meta::speakerCount = 0
# meta::summary1 = (human) This paper presents an eye tracking study to explore how viewers experience art, focusing on a 17th Century collection of Spanish paintings by Zurbarán.
# meta::summary2 = (claude-3-5-sonnet-20241022) This pilot study uses eye-tracking techniques to examine how viewers visually pro

As you can see, this is a tabular format. There is a line for each word in the sentence, and the information for that word is given in tab-delimited cells, for example:

```2	Appreciation	appreciation	NOUN	NN	Number=Sing	0	root	0:root	Entity=1)|MSeg=Appreciat-ion```

This is the CoNLL-U format, which originated with a shared task at the Conference on Natural Language Learning (CoNLL). 

There is a Python package, conllu, that is made for reading CoNLL-U data. Once we have read the GUM corpus into a string, we can parse it with conllu:

In [ ]:
sentences = conllu.parse(data)

The content of `sentences` is a sequence of TokenList objects. Here is the one for the 10th sentence of the corpus:

In [ ]:
print(sentences[10])

TokenList<Thus, ,, the, time, it, takes, and, the, ways, of, visually, exploring, an, artwork, can, inform, about, its, relevance, ,, interestingness, ,, and, even, its, aesthetic, appeal, ., metadata={sent_id: "GUM_academic_art-11", s_type: "sub", s_prominence: "3", transition: "establishment", text: "Thus, the time it takes and the ways of visually exploring an artwork can inform about its relevance, interestingness, and even its aesthetic appeal."}>


We can access the entries on the TokenList through a for-loop, or using an index. Here is the first token of sentence 10. As you can see, it is a Python dictionary.

In [ ]:
sentence10 = sentences[10]
firstword = sentence10[0]
firstword

{'id': 1,
 'form': 'Thus',
 'lemma': 'thus',
 'upos': 'ADV',
 'xpos': 'RB',
 'feats': None,
 'head': 16,
 'deprel': 'advmod',
 'deps': [('advmod', 16)],
 'misc': {'Discourse': 'context-background:12->23:7:_',
  'PDTB': 'Explicit:Contingency.Cause.Result:thus:107:79-106:108-134',
  'SpaceAfter': 'No'}}

You can access the entries in that dictionary by their keys:

In [ ]:
firstword["lemma"]

'thus'

To better understand this big dictionary, it helps to view it as an attribute-value matrix. Here is the first word of the 10th sentence of the UD_English-GUM corpus:

In [ ]:
firstword = {'id': 1,
  'form': 'Thus',
  'lemma': 'thus',
  'upos': 'ADV',
  'xpos': 'RB',
  'feats': None,
  'head': 16,
  'deprel': 'advmod',
  'deps': None,
  'misc': {'SpaceAfter': 'No'}}

This is the following attribute-value matrix (AVM):

$$
\left[\begin{array}{ll}
\text{id:} & 1\\
\text{form:} & 'Thus'\\
\text{lemma:} & 'thus'\\
\text{upos:} &  'ADV'\\
\text{xpos:} & 'RB'\\
\text{feats:} &  None\\
\text{head:} & 16\\
\text{deprel:}  & advmod\\
\text{deps:}  & None\\
\text{misc:} & \left[\begin{array}{ll}
\text{SpaceAfter:} & 'No'
\end{array}\right]
\end{array}\right]
$$

As you saw above, you can access an entry in this attribute-value matrix through its dictionary key:

In [ ]:
firstword["lemma"]

'thus'

One of the values in the AVM is itself an AVM. To access the value that tells you whether there is a space after the word, you need to specify the whole path of keys. `firstword["misc"]` accesses a dictionary, namely `{'SpaceAfter': 'No'}`, which again has keys, in particular `SpaceAfter`: 

In [ ]:
firstword["misc"]["SpaceAfter"]

'No'

The Universal Dependencies representation of a whole sentence is a list of tokens, that is, a list of dictionaries (=AVMs):

In [ ]:
sentence10 = [{'id': 1,
  'form': 'Thus',
  'lemma': 'thus',
  'upos': 'ADV',
  'xpos': 'RB',
  'feats': None,
  'head': 16,
  'deprel': 'advmod',
  'deps': None,
  'misc': {'SpaceAfter': 'No'}},
 {'id': 2,
  'form': ',',
  'lemma': ',',
  'upos': 'PUNCT',
  'xpos': ',',
  'feats': None,
  'head': 1,
  'deprel': 'punct',
  'deps': None,
  'misc': None},
 {'id': 3,
  'form': 'the',
  'lemma': 'the',
  'upos': 'DET',
  'xpos': 'DT',
  'feats': {'Definite': 'Def', 'PronType': 'Art'},
  'head': 4,
  'deprel': 'det',
  'deps': None,
  'misc': None},
 {'id': 4,
  'form': 'time',
  'lemma': 'time',
  'upos': 'NOUN',
  'xpos': 'NN',
  'feats': {'Number': 'Sing'},
  'head': 16,
  'deprel': 'nsubj',
  'deps': None,
  'misc': None},
 {'id': 5,
  'form': 'it',
  'lemma': 'it',
  'upos': 'PRON',
  'xpos': 'PRP',
  'feats': {'Case': 'Nom',
   'Gender': 'Neut',
   'Number': 'Sing',
   'Person': '3',
   'PronType': 'Prs'},
  'head': 6,
  'deprel': 'nsubj',
  'deps': None,
  'misc': None},
 {'id': 6,
  'form': 'takes',
  'lemma': 'take',
  'upos': 'VERB',
  'xpos': 'VBZ',
  'feats': {'Mood': 'Ind',
   'Number': 'Sing',
   'Person': '3',
   'Tense': 'Pres',
   'VerbForm': 'Fin'},
  'head': 4,
  'deprel': 'acl:relcl',
  'deps': None,
  'misc': None},
 {'id': 7,
  'form': 'and',
  'lemma': 'and',
  'upos': 'CCONJ',
  'xpos': 'CC',
  'feats': None,
  'head': 9,
  'deprel': 'cc',
  'deps': None,
  'misc': None},
 {'id': 8,
  'form': 'the',
  'lemma': 'the',
  'upos': 'DET',
  'xpos': 'DT',
  'feats': {'Definite': 'Def', 'PronType': 'Art'},
  'head': 9,
  'deprel': 'det',
  'deps': None,
  'misc': None},
 {'id': 9,
  'form': 'ways',
  'lemma': 'way',
  'upos': 'NOUN',
  'xpos': 'NNS',
  'feats': {'Number': 'Plur'},
  'head': 4,
  'deprel': 'conj',
  'deps': None,
  'misc': None},
 {'id': 10,
  'form': 'of',
  'lemma': 'of',
  'upos': 'SCONJ',
  'xpos': 'IN',
  'feats': None,
  'head': 12,
  'deprel': 'mark',
  'deps': None,
  'misc': None},
 {'id': 11,
  'form': 'visually',
  'lemma': 'visually',
  'upos': 'ADV',
  'xpos': 'RB',
  'feats': None,
  'head': 12,
  'deprel': 'advmod',
  'deps': None,
  'misc': None},
 {'id': 12,
  'form': 'exploring',
  'lemma': 'explore',
  'upos': 'VERB',
  'xpos': 'VBG',
  'feats': {'VerbForm': 'Ger'},
  'head': 9,
  'deprel': 'acl',
  'deps': None,
  'misc': None},
 {'id': 13,
  'form': 'an',
  'lemma': 'a',
  'upos': 'DET',
  'xpos': 'DT',
  'feats': {'Definite': 'Ind', 'PronType': 'Art'},
  'head': 14,
  'deprel': 'det',
  'deps': None,
  'misc': None},
 {'id': 14,
  'form': 'artwork',
  'lemma': 'artwork',
  'upos': 'NOUN',
  'xpos': 'NN',
  'feats': {'Number': 'Sing'},
  'head': 12,
  'deprel': 'obj',
  'deps': None,
  'misc': None},
 {'id': 15,
  'form': 'can',
  'lemma': 'can',
  'upos': 'AUX',
  'xpos': 'MD',
  'feats': {'VerbForm': 'Fin'},
  'head': 16,
  'deprel': 'aux',
  'deps': None,
  'misc': None},
 {'id': 16,
  'form': 'inform',
  'lemma': 'inform',
  'upos': 'VERB',
  'xpos': 'VB',
  'feats': {'VerbForm': 'Inf'},
  'head': 0,
  'deprel': 'root',
  'deps': None,
  'misc': None},
 {'id': 17,
  'form': 'about',
  'lemma': 'about',
  'upos': 'ADP',
  'xpos': 'IN',
  'feats': None,
  'head': 19,
  'deprel': 'case',
  'deps': None,
  'misc': None},
 {'id': 18,
  'form': 'its',
  'lemma': 'its',
  'upos': 'PRON',
  'xpos': 'PRP$',
  'feats': {'Gender': 'Neut',
   'Number': 'Sing',
   'Person': '3',
   'Poss': 'Yes',
   'PronType': 'Prs'},
  'head': 19,
  'deprel': 'nmod:poss',
  'deps': None,
  'misc': None},
 {'id': 19,
  'form': 'relevance',
  'lemma': 'relevance',
  'upos': 'NOUN',
  'xpos': 'NN',
  'feats': {'Number': 'Sing'},
  'head': 16,
  'deprel': 'obl',
  'deps': None,
  'misc': {'SpaceAfter': 'No'}},
 {'id': 20,
  'form': ',',
  'lemma': ',',
  'upos': 'PUNCT',
  'xpos': ',',
  'feats': None,
  'head': 21,
  'deprel': 'punct',
  'deps': None,
  'misc': None},
 {'id': 21,
  'form': 'interestingness',
  'lemma': 'interestingness',
  'upos': 'NOUN',
  'xpos': 'NN',
  'feats': {'Number': 'Sing'},
  'head': 19,
  'deprel': 'conj',
  'deps': None,
  'misc': {'SpaceAfter': 'No'}},
 {'id': 22,
  'form': ',',
  'lemma': ',',
  'upos': 'PUNCT',
  'xpos': ',',
  'feats': None,
  'head': 27,
  'deprel': 'punct',
  'deps': None,
  'misc': None},
 {'id': 23,
  'form': 'and',
  'lemma': 'and',
  'upos': 'CCONJ',
  'xpos': 'CC',
  'feats': None,
  'head': 27,
  'deprel': 'cc',
  'deps': None,
  'misc': None},
 {'id': 24,
  'form': 'even',
  'lemma': 'even',
  'upos': 'ADV',
  'xpos': 'RB',
  'feats': None,
  'head': 27,
  'deprel': 'advmod',
  'deps': None,
  'misc': None},
 {'id': 25,
  'form': 'its',
  'lemma': 'its',
  'upos': 'PRON',
  'xpos': 'PRP$',
  'feats': {'Gender': 'Neut',
   'Number': 'Sing',
   'Person': '3',
   'Poss': 'Yes',
   'PronType': 'Prs'},
  'head': 27,
  'deprel': 'nmod:poss',
  'deps': None,
  'misc': None},
 {'id': 26,
  'form': 'aesthetic',
  'lemma': 'aesthetic',
  'upos': 'ADJ',
  'xpos': 'JJ',
  'feats': {'Degree': 'Pos'},
  'head': 27,
  'deprel': 'amod',
  'deps': None,
  'misc': None},
 {'id': 27,
  'form': 'appeal',
  'lemma': 'appeal',
  'upos': 'NOUN',
  'xpos': 'NN',
  'feats': {'Number': 'Sing'},
  'head': 19,
  'deprel': 'conj',
  'deps': None,
  'misc': {'SpaceAfter': 'No'}},
 {'id': 28,
  'form': '.',
  'lemma': '.',
  'upos': 'PUNCT',
  'xpos': '.',
  'feats': None,
  'head': 16,
  'deprel': 'punct',
  'deps': None,
  'misc': None}]

In [ ]:
# now we can iterate through the AVMs for this sentence, and 
# print informati0n for each one
for token in sentence10:
    print(token["id"], token["form"], token["upos"], 
          token["head"], token["deprel"])

1 Thus ADV 16 advmod
2 , PUNCT 1 punct
3 the DET 4 det
4 time NOUN 16 nsubj
5 it PRON 6 nsubj
6 takes VERB 4 acl:relcl
7 and CCONJ 9 cc
8 the DET 9 det
9 ways NOUN 4 conj
10 of SCONJ 12 mark
11 visually ADV 12 advmod
12 exploring VERB 9 acl
13 an DET 14 det
14 artwork NOUN 12 obj
15 can AUX 16 aux
16 inform VERB 0 root
17 about ADP 19 case
18 its PRON 19 nmod:poss
19 relevance NOUN 16 obl
20 , PUNCT 21 punct
21 interestingness NOUN 19 conj
22 , PUNCT 27 punct
23 and CCONJ 27 cc
24 even ADV 27 advmod
25 its PRON 27 nmod:poss
26 aesthetic ADJ 27 amod
27 appeal NOUN 19 conj
28 . PUNCT 16 punct


Now say we want to determine how often we have subject-verb-object (SVO) versus SOV versus VSO etc. in a Universal Dependencies corpus. To do that, we would like to have an AVM for a word that includes all its dependents. For the verb "inform" in the sentence above, we would like the AVM to list that "time" (word 4) is the nsubj of "inform", and "relevance" (word 19) is its obl:


$$
\left[\begin{array}{ll}
\text{form:} & inform\\
\text{id:} & 16\\
\text{upos:} & VERB\\
\text{dep:} & \[ \left[\begin{array}{ll}
\text{id:} & 4\\
\text{deprel:} & nsubj\end{array}\right], 
\left[\begin{array}{ll}
\text{id:} & 19\\
\text{deprel:} & obl\end{array}\right]\]
\end{array}\right]
$$


As a Python data structure, this AVM is rather complex: It is a dictionary, but under the key "dep" the value is a list of dictionaries. 


In [ ]:
inform_avm_with_deps = { "form" : "inform",
                        "id" : 16,
                        "upos" : "VERB",
                        "dep" : [ {"id" : 4, "deprel" : "nsubj"}, 
                                  {"id" : 19, "deprel" : "obl"}]
                       }

Here is how we make a version of sentence 10 that has such an AVM for each word.

In [ ]:
def reformat_sentence(sentence):
    # first we initialize each AVM to have an empty dependencies list
    sentence_reformatted = [ ]
    for token in sentence:
        sentence_reformatted.append( { "form" : token["form"], 
                                    "id" : token["id"],
                                    "upos" : token["upos"],
                                    "dep" : [ ]
                                  } )

    # now we add dependencies
    for token in sentence:
        # looking up the head of this token. index is that head minus one.
        myhead_ix = token["head"] - 1
        # print(token["form"], token["id"], token["head"], sentence10_reformat[myhead_ix]["form"])
        # adding this token to the head's dependencies
        sentence_reformatted[ myhead_ix ]["dep"].append({ "id" : token["id"],
                                                       "deprel" : token["deprel"]})

    return sentence_reformatted
    
reformat_sentence(sentence10)

[{'form': 'Thus',
  'id': 1,
  'upos': 'ADV',
  'dep': [{'id': 2, 'deprel': 'punct'}]},
 {'form': ',', 'id': 2, 'upos': 'PUNCT', 'dep': []},
 {'form': 'the', 'id': 3, 'upos': 'DET', 'dep': []},
 {'form': 'time',
  'id': 4,
  'upos': 'NOUN',
  'dep': [{'id': 3, 'deprel': 'det'},
   {'id': 6, 'deprel': 'acl:relcl'},
   {'id': 9, 'deprel': 'conj'}]},
 {'form': 'it', 'id': 5, 'upos': 'PRON', 'dep': []},
 {'form': 'takes',
  'id': 6,
  'upos': 'VERB',
  'dep': [{'id': 5, 'deprel': 'nsubj'}]},
 {'form': 'and', 'id': 7, 'upos': 'CCONJ', 'dep': []},
 {'form': 'the', 'id': 8, 'upos': 'DET', 'dep': []},
 {'form': 'ways',
  'id': 9,
  'upos': 'NOUN',
  'dep': [{'id': 7, 'deprel': 'cc'},
   {'id': 8, 'deprel': 'det'},
   {'id': 12, 'deprel': 'acl'}]},
 {'form': 'of', 'id': 10, 'upos': 'SCONJ', 'dep': []},
 {'form': 'visually', 'id': 11, 'upos': 'ADV', 'dep': []},
 {'form': 'exploring',
  'id': 12,
  'upos': 'VERB',
  'dep': [{'id': 10, 'deprel': 'mark'},
   {'id': 11, 'deprel': 'advmod'},
   {'id'

Based on this data structure, we can determine whether the subject is before the verb: If so, its ID is lower than that of the verb. We can also determine whether the subject is before the object: If so, its ID is lower than that of the the object.

We can also see how far away from the verb the subject is, by computing the difference between the IDs of the verb and its subject. In the same way, we can determine how far away from the verb the direct object is. 